<a href="https://colab.research.google.com/github/DaeSeokSong/MachineLearningModels/blob/main/LSTM_PRVS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Price Rationality Verification System

* 가격 합리성 검증 시스템

### Google Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/DeepLearning/Project/PRVS
!ls -al

Mounted at /content/gdrive
/content/gdrive/MyDrive/DeepLearning/Project/PRVS
total 426
drwx------ 2 root root   4096 Jan 23 02:35 Dataset
-rw------- 1 root root 431637 Jan 29 02:02 LSTM-PRVS.ipynb


### Install

In [2]:
!pip install beautifulsoup4

### Import

In [48]:
# Crawling
import requests
from bs4 import BeautifulSoup

# Modeling
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import metrics, losses
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

### Global Variable

### Main

In [53]:
# 나라장터 가격 크롤링 불가, 배추김치(포기김치, 송이식품) 1kg 가격
price_go = 3500

# 네이버 가격 크롤링
url_naver = 'https://search.shopping.naver.com/search/all?where=all&frm=NVSCTAB&query=%EB%B0%B0%EC%B6%94%EA%B9%80%EC%B9%98%2C+%EC%86%A1%EC%9D%B4%EC%8B%9D%ED%92%88%2C+%EB%A7%9B%EA%B9%80%EC%B9%98%2C+1kg'
response = requests.get(url_naver)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    ul = soup.select_one('#__next > div > div.style_container__1YjHN > div')

    prices = ul.select('div > div > ul > div > div > li > div > div > div > strong > span > span')
    for idx, price in enumerate(prices) : 
        prices[idx] = int(price.get_text().split(',')[0] + price.get_text().split(',')[1][:-1])

    prices = np.array(prices)
    print(prices)
    mean = np.mean(prices)
    std = np.std(prices)
    cov = std / mean * 100
    print('평균 = ', mean)
    print('표준편차 = ', std)
    print('변동 계수 = ', cov)

    print('\n########## 최대값 제거 ##########')
    prices = np.delete(prices, np.where(prices == max(prices)))
    print(prices)
    mean = np.mean(prices)
    std = np.std(prices)
    cov = std / mean * 100
    print('평균 = ', mean)
    print('표준편차 = ', std)
    print('변동 계수 = ', cov)

    print('\n########## 최소값 제거 ##########')
    prices = np.delete(prices, np.where(prices == min(prices)))
    print(prices)
    mean = np.mean(prices)
    std = np.std(prices)
    cov = std / mean * 100
    print('평균 = ', mean)
    print('표준편차 = ', std)
    print('변동 계수 = ', cov)

    print('\n########## 결과값 ##########')
    mean = int(round(np.mean(prices), -1))
    print('나라장터 가격 = ', price_go)
    print('네이버쇼핑 평균가 = ', mean)
    print('가격 차이 = ', abs(price_go - mean))
else : 
    print(response.status_code)

[ 7700  9900 13000 13900 25900]
평균 =  14080.0
표준편차 =  6310.118857834613
변동 계수 =  44.8161850698481

########## 최대값 제거 ##########
[ 7700  9900 13000 13900]
평균 =  11125.0
표준편차 =  2472.2206616724166
변동 계수 =  22.22220819480824

########## 최소값 제거 ##########
[ 9900 13000 13900]
평균 =  12266.666666666666
표준편차 =  1713.3463034528531
변동 계수 =  13.967497039017825

########## 결과값 ##########
나라장터 가격 =  3500
네이버쇼핑 평균가 =  12270
가격 차이 =  8770


In [ ]:
df = pd.read_csv('./Dataset/item_registration_details.csv', index_col=0)
df.head()